In [ ]:
ruta_archivos = r"C:\Users\Home\Desktop\Proyecto Hackathon Copernicus"
os.chdir(ruta_archivos)
os.getcwd()

In [ ]:
lista = glob.glob("ImagenRaster.tif")
lista

In [ ]:
import rasterio
import numpy as np

In [ ]:
SENTINEL2_RS = rasterio.open(lista[0])

In [ ]:
import geopandas as gpd

In [ ]:
ROI = gpd.read_file(r'C:\Users\Home\Desktop\Proyecto Hackathon Copernicus\ROI.shp')

In [ ]:
# Proyección de datos
print("Proyección ROI:",roi.crs)
print("Proyección imagen:",S2_RS.crs)
print("Numero de bandas imagen:",S2_RS.count)

In [ ]:
Proyección ROI: epsg:32719
Proyección imagen: EPSG:32719
Numero de bandas imagen: 11

In [ ]:
# lista coberturas para el analisis
lista_cobertura = []
for point in roi['Class']:
    lista_cobertura.append(point[0:50])

In [ ]:
# Creacion de lista de coordenada Norte y Este
lista_este = []
lista_norte = []
for point in roi['geometry']:
    lista_este.append(round(point.xy[0][0],1))
    lista_norte.append(round(point.xy[1][0],1))

In [ ]:
lista_banda1 = []
lista_banda2 = []
lista_banda3 = []
lista_banda4 = []
lista_banda5 = []
lista_banda6 = []
lista_banda7 = []
lista_banda8 = []
lista_banda9 = []
lista_banda11 = []
lista_banda12 = []
for point in roi['geometry']:
    x = point.xy[0][0]
    y = point.xy[1][0]
    row, col = S2_RS.index(x,y)
    lista_banda1.append(SENTINEL2_RS.read(1)[row, col])
    lista_banda2.append(SENTINEL2_RS.read(2)[row, col])
    lista_banda3.append(SENTINEL2_RS.read(3)[row, col])
    lista_banda4.append(SENTINEL2_RS.read(4)[row, col])
    lista_banda5.append(SENTINEL2_RS.read(5)[row, col])
    lista_banda6.append(SENTINEL2_RS.read(6)[row, col])
    lista_banda7.append(SENTINEL2_RS.read(7)[row, col])
    lista_banda8.append(SENTINEL2_RS.read(8)[row, col])
    lista_banda9.append(SENTINEL2_RS.read(9)[row, col])
    lista_banda11.append(SENTINEL2_RS.read(10)[row, col])
    lista_banda12.append(SENTINEL2_RS.read(11)[row, col])

In [ ]:
# Creacion de diccionario de datos
datos = {"Cobertura": lista_cobertura, 
         "Este": lista_este,
         "Norte": lista_norte,
         "B1": lista_banda1,
         "B2": lista_banda2,
         "B3": lista_banda3,
         "B4": lista_banda4,
         "B5": lista_banda5,
         "B6": lista_banda6,
         "B7": lista_banda7,
         "B8": lista_banda8,
         "B9": lista_banda9,
         "B11": lista_banda11,
         "B12": lista_banda12}

In [ ]:
import pandas as pd

In [ ]:
# Creacion de DataFrame mediante el diccionario anterior
dataframe = pd.DataFrame(datos)
dataframe.head()

In [ ]:
#SEPARACION DE VARIABLES PARA LA PREDICCION DEL MODELO

In [ ]:
X = dataframe.iloc[:,3:14]
print(X.head())

In [ ]:
y = dataframe.iloc[:,0]
print(y.head())

In [ ]:
# SEPARACION DE LOS DATOS DE ENTRENAMIENTO Y VALIDACION 

In [ ]:
from sklearn.model_selection import train_test_split # Entrenamiento y validacion 
from sklearn.metrics import confusion_matrix 

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state = 0)

In [ ]:
Xtrain.head(), Xtest.head()

In [ ]:
#MODELO RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RF

In [ ]:
instancia_rf = RF()
print(instancia_rf)

In [ ]:
instancia_rf.fit(Xtrain, ytrain)

In [ ]:
print("las predicciones en xtest son:{}".format(instancia_rf.predict(Xtest)))

In [ ]:
print(" precision en testing: {:.3f}".format(instancia_rf.score(Xtest, ytest)))

In [ ]:
prediccion = instancia_rf.predict(Xtest)
MC = confusion_matrix(ytest, prediccion)
print('Matriz de confusión:\n{}'.format(MC))

In [ ]:
#APLICANDO EL MODELO A TODA LA IMAGEN TENEMOS

In [ ]:
mat = np.dstack((S2_RS.read(1),S2_RS.read(2),S2_RS.read(3),S2_RS.read(4),S2_RS.read(5),S2_RS.read(6),
                 S2_RS.read(7),S2_RS.read(8),S2_RS.read(9),S2_RS.read(10),S2_RS.read(11)))
mat.shape

In [ ]:
rows, cols, nbands = mat.shape
rows, cols, nbands

In [ ]:
mat = mat.reshape((rows*cols, nbands)) #mat = mat.reshape((-11, 11))
mat.shape

In [ ]:
resultados = instancia_rf.predict(mat)
rf = resultados.reshape((rows, cols))
rf

In [ ]:
#Cambiar de STR a INT segun los nombres de las clasificaciones 
rf[rf == 'Clasificacion 1'] = 1
rf[rf == 'Clasificacion 2'] = 2
rf[rf == 'Clasificacion 3'] = 3
rf[rf == 'Clasificacion 4'] = 4
rf

In [ ]:
# Covertir a tipo INT
rf = rf.astype(int)
rf

In [ ]:
#VISUALIZACION DE DATOS 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(8,7))
plt.imshow(rf, cmap="RdYlGn_r") # jet, Spectral, viridis, RdYlGn, hsv
plt.colorbar()
plt.title("Sentinel-2 Clasificación supervisada: Random Forest") # titulo
plt.xlabel("Numero de Columna") # etiqueta X
plt.ylabel("Numero de Fila") # etiqueta Y
plt.show()